# CMRM Assignment No. 2

In [ ]:
import os
import librosa
import scipy
import sklearn # pip install sklearn
import numpy as np
import pandas as pd # pip install pandas
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm # pip install tqdm
from sklearn.linear_model import LogisticRegression
from mutagen.easyid3 import EasyID3 # pip install mutagen

## Question 1

In [ ]:
train_path = 'sonics/train'
test_path = 'sonics/test'

# -------------------------------------------------- #
# Train set
fake_path = train_path + '/fake'
real_path = train_path + '/real'

fake_train = [i for i in os.listdir(fake_path) if i[0] != '.']
fake_train.sort()

# Print the number of files
print("Number of fake tracks in the train set:", len(fake_train))


real_train = [i for i in os.listdir(real_path) if i[0] != '.']
real_train.sort()

# Print the number of files
print("Number of real tracks in the train set:", len(real_train))




# -------------------------------------------------- #
# Test set

test_set = [i for i in os.listdir(test_path) if i[0] != '.']
test_set.sort()

# Print the number of files
print("Number of tracks in the test set:", len(test_set))

In [ ]:
# Load signal fake_train[0] - Time domain
Fs = 22050
# Fs_fake = 22050 / 2
# Fs_real = 44100
def print_plot_play(x, Fs, text=''):
    print('%s Fs = %d, x.shape = %s, x.dtype = %s' % (text, Fs, x.shape, x.dtype))
    plt.figure(figsize=(10, 2))
    plt.plot(x, color='gray')
    plt.xlim([0, x.shape[0]])
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')
    plt.tight_layout()
    plt.show()
    ipd.display(ipd.Audio(data=x, rate=Fs))
    


#Fake track 1
fake_mp3_1_path = os.path.join(fake_path, fake_train[0])
fake_mp3_1, _ = librosa.load(fake_mp3_1_path, sr=Fs)

print_plot_play(x=fake_mp3_1, Fs=Fs, text='First Fake MP3: ')

#Real track 1
real_mp3_1_path = os.path.join(real_path, real_train[0])
real_mp3_1, _ = librosa.load(real_mp3_1_path, sr=Fs)

print_plot_play(x=real_mp3_1, Fs=Fs, text='First Real MP3: ')

In [ ]:
# FFT
#use np.fft
#Fake song 1
X = np.abs(np.fft.fft(fake_mp3_1))
N = fake_mp3_1.shape[0] #number of samples
freq = np.fft.fftfreq(N, d=1/Fs)
X = X[:N//2]
freq = freq[:N//2]

print("Fake song 1 Frequency Spectrum")
plt.figure(figsize=(15, 2))
plt.plot(freq, X, c='k')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()

#Real song 1
X = np.abs(np.fft.fft(real_mp3_1))
N = real_mp3_1.shape[0] #number of samples
freq = np.fft.fftfreq(N, d=1/Fs)
X = X[:N//2]
freq = freq[:N//2]

print("Real song 1 Frequency Spectrum")
plt.figure(figsize=(15, 2))
plt.plot(freq, X, c='k')
#plt.xlim([20, 10000])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()

# STFT
N = 4096
H = 1024

def stft_basic(x, w, H=8):
    """Compute a basic version of the discrete short-time Fourier transform (STFT)

    Args:
        x: Signal to be transformed
        w: Window function
        H: Hopsize

    Returns:
        X: The discrete short-time Fourier transform
    """
    N = len(w)
    L = len(x)
    M = np.floor((L - N) / H).astype(int)
    X = np.zeros((N, M + 1), dtype='complex')
    for m in range(M + 1):
        x_win = x[m * H:m * H + N] * w
        X_win = np.fft.fft(x_win)
        X[:, m] = X_win
    K = (N + 1) // 2
    X = X[:K, :]
    return X

w = np.hanning(N)
X_fake = stft_basic(fake_mp3_1, w, H)
X_real = stft_basic(real_mp3_1, w, H)


def plot_power_spectrogram(x, X, N, Fs):
    Y = np.abs(X) ** 2
    eps = np.finfo(float).eps

    Y_db = 10 * np.log10(Y + eps)
    real_t = np.arange(len(x)) / Fs

    plt.figure(figsize=(12, 4))

    plt.plot(real_t, x)
    plt.xlim([min(real_t), max(real_t)])

    time_axis = np.arange(X.shape[1]) * H / Fs
    frequency_axis = np.arange(X.shape[0]) * Fs / N

    left = min(time_axis)
    right = max(time_axis) + N / Fs
    lower = min(frequency_axis)
    upper = max(frequency_axis)
    plt.figure(figsize=(12, 4))
    im = plt.imshow(Y_db, origin='lower', aspect='auto', cmap='gray_r', extent=[left, right, lower, upper], vmin=-30,
                    vmax=20)
    #im = plt.imshow(Y_db, origin='lower', aspect='auto', cmap='gray_r', vmin=-30, vmax=20)
    plt.colorbar(im)
    plt.ylim([0, 5000])
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plt.show()


#Plot power "Spectrogram"...not frequency spectrum graph
print("Fake song 1 STFT Power Spectrogram")
plot_power_spectrogram(fake_mp3_1, X_fake, N, Fs)

#Plot real
print("Real song 1 STFT Power Spectrogram")
plot_power_spectrogram(real_mp3_1, X_real, N, Fs)


## Question 2

In [ ]:
from utils import lower_envelope, max_normalise, curve_profile

def fakeprint(stft, f_range = [0, 16000], fs = 22050):
    """
    Compute the fakeprint feature from an STFT representation.

        Args:
            stft: 2D STFT spectrogram.
            f_range: Frequency range [min_freq, max_freq] over which the
                     curve profile is computed.
            SR: Sampling rate of the original audio signal.

        Returns:
            fp_curve: Normalized fakeprint feature curve extracted from the
                      averaged spectral profile.
    """

    
    return fp_curve

In [ ]:
# Test fakeprint



## Question 3

In [ ]:
def compute_feature_vector(filename, Fs, N=4096, H=1024):
    """Compute the feature vector 

    Args:
        filename: Input filename
        Fs: Sampling rate
        N: Window length
        H: Hop size

    Returns:
        f_vector: Feature vector
    """
    
    
    return f_vector

In [ ]:
# Define labels_dict

# Compute feature vector - train

# Compute feature vector - test


In [ ]:
# Train regressor


In [ ]:
# Print the accuracy for the train set

# Plot the confusion matrix


## Question 4

In [ ]:
# Compute prediction for the test set

# Print tracks classified as 'real'
real_tracks = []
print('The real tracks are: \n')


In [ ]:
# Find password
psw = 0


print(f'The password is: {psw}')

In [ ]:
# Verify if your items are real



In [ ]:
# Print the accuracy for the test set

# Plot the confusion matrix

## Question 5

In [ ]:
# Find the key sentence
